# Classificação de Imagens com CNN (PyTorch): Cães vs. Gatos 🐶🐱

**Objetivo:** Construir um modelo de Inteligência Artificial com Redes Neurais Convolucionais (CNN) para classificar imagens de cachorros e gatos, utilizando a biblioteca **PyTorch** como alternativa ao TensorFlow/Keras.

**Dataset:** [Dogs vs. Cats - Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data)

**Estrutura do Projeto:**
1.  **Preparação dos Dados:** Carregamento, organização dos diretórios, pré-processamento e aumento de dados com `torchvision`.
2.  **Construção e Treinamento:** Definição da arquitetura da CNN em PyTorch, escrita do loop de treinamento manual e treinamento do modelo.
3.  **Avaliação e Testes:** Análise do desempenho com métricas (precisão, recall, F1-score) e teste com novas imagens.
4.  **Conclusão:** Resumo dos resultados e próximos passos.

## 1. Preparação dos Dados

### 1.1. Importação das Bibliotecas e Definição dos Caminhos

Importamos as bibliotecas necessárias, incluindo `torch` e `torchvision`. A definição dos caminhos e a organização dos arquivos permanecem as mesmas.

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn.metrics import classification_report, confusion_matrix

# Caminho para o diretório de treino original do Kaggle
# ATENÇÃO: Altere este caminho para o local onde você descompactou a pasta 'train'
original_train_dir = './train/'

# Caminho para o novo diretório base onde organizaremos os dados
base_dir = './cats_vs_dogs_data/'

### 1.2. Criação dos Diretórios e Separação dos Dados (70/15/15)

Esta etapa é idêntica à versão com TensorFlow. Criamos uma nova estrutura de diretórios (`train`, `validation`, `test`) e movemos as imagens para seus respectivos locais, garantindo que `ImageFolder` do PyTorch possa encontrá-las corretamente.

In [ ]:
# Cria o diretório base se não existir
if os.path.exists(base_dir):
    shutil.rmtree(base_dir) # Remove o diretório se já existir para começar do zero
os.makedirs(base_dir)

# Cria os subdiretórios de treino, validação e teste
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir)

# Cria os diretórios de classes (gatos e cachorros) dentro de cada conjunto
for dir_path in [train_dir, validation_dir, test_dir]:
    os.makedirs(os.path.join(dir_path, 'cats'))
    os.makedirs(os.path.join(dir_path, 'dogs'))

# Nomes dos arquivos originais
all_fnames = os.listdir(original_train_dir)
cat_fnames = [fname for fname in all_fnames if fname.startswith('cat')]
dog_fnames = [fname for fname in all_fnames if fname.startswith('dog')]

# Função para copiar as imagens para os novos diretórios
def copy_files(fnames, source_dir, dest_dirs):
    for fname, dest in zip(fnames, dest_dirs):
        src = os.path.join(source_dir, fname)
        dst = os.path.join(dest, fname)
        shutil.copyfile(src, dst)

# Dividindo gatos (70-15-15)
np.random.shuffle(cat_fnames)
train_split = int(0.7 * len(cat_fnames))
val_split = int(0.15 * len(cat_fnames))
cat_train = cat_fnames[:train_split]
cat_val = cat_fnames[train_split:train_split+val_split]
cat_test = cat_fnames[train_split+val_split:]
copy_files(cat_train, original_train_dir, [os.path.join(train_dir, 'cats')] * len(cat_train))
copy_files(cat_val, original_train_dir, [os.path.join(validation_dir, 'cats')] * len(cat_val))
copy_files(cat_test, original_train_dir, [os.path.join(test_dir, 'cats')] * len(cat_test))

# Dividindo cachorros (70-15-15)
np.random.shuffle(dog_fnames)
train_split = int(0.7 * len(dog_fnames))
val_split = int(0.15 * len(dog_fnames))
dog_train = dog_fnames[:train_split]
dog_val = dog_fnames[train_split:train_split+val_split]
dog_test = dog_fnames[train_split+val_split:]
copy_files(dog_train, original_train_dir, [os.path.join(train_dir, 'dogs')] * len(dog_train))
copy_files(dog_val, original_train_dir, [os.path.join(validation_dir, 'dogs')] * len(dog_val))
copy_files(dog_test, original_train_dir, [os.path.join(test_dir, 'dogs')] * len(dog_test))

print(f"Total de imagens de treino: {len(cat_train) + len(dog_train)}")
print(f"Total de imagens de validação: {len(cat_val) + len(dog_val)}")
print(f"Total de imagens de teste: {len(cat_test) + len(dog_test)}")

### 1.3. Pré-processamento e Carregamento dos Dados com `torchvision`

Em vez do `ImageDataGenerator`, usamos `torchvision.transforms` para definir as operações de pré-processamento e aumento de dados. Em seguida, usamos `datasets.ImageFolder` para ler as imagens da nossa estrutura de diretórios e `DataLoader` para criar lotes (batches) de dados de forma eficiente.

In [ ]:
IMG_SIZE = 150
BATCH_SIZE = 32

# Transformações para o conjunto de treino (com Data Augmentation)
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # Converte para tensor e normaliza para [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalização padrão da ImageNet
])

# Transformações para os conjuntos de validação e teste (sem Data Augmentation)
val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Criando os Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(validation_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# Criando os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Classes encontradas: {train_dataset.classes}")
# ImageFolder atribui 'cats' a 0 e 'dogs' a 1 por ordem alfabética.

## 2. Construção e Treinamento de uma CNN

### 2.1. Criação da Rede Convolucional em PyTorch

Em PyTorch, definimos o modelo como uma classe que herda de `nn.Module`. As camadas são definidas no construtor `__init__`, e a ordem de execução (o *forward pass*) é definida no método `forward`.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # Camada 1: Entrada (3, 150, 150)
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Camada 2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Camada 3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.flatten = nn.Flatten()
        
        self.fc_layers = nn.Sequential(
            # O cálculo do tamanho de entrada é 128 * (150/2/2/2) * (150/2/2/2) = 128 * 18 * 18 = 41472
            nn.Linear(128 * 18 * 18, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1) # Saída única para classificação binária
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc_layers(x)
        return torch.sigmoid(x) # Aplicamos a sigmoide na saída

# Instala a biblioteca para visualizar o resumo (opcional, mas útil)
# !pip install torchinfo
from torchinfo import summary

model = SimpleCNN()
summary(model, input_size=(BATCH_SIZE, 3, IMG_SIZE, IMG_SIZE))

### 2.2. Treinamento do Modelo

Definimos a função de perda (`BCELoss` para entropia cruzada binária), o otimizador (`RMSprop`) e o dispositivo (GPU, se disponível). Em seguida, escrevemos o loop de treinamento manual, que itera sobre as épocas e os lotes de dados, calcula a perda, faz a retropropagação (backpropagation) e atualiza os pesos do modelo.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
model.to(device)

criterion = nn.BCELoss() # Binary Cross-Entropy Loss
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)

EPOCHS = 20

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(EPOCHS):
    # --- Treinamento ---
    model.train() # Coloca o modelo em modo de treino
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
        
        optimizer.zero_grad() # Zera os gradientes
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward() # Backpropagation
        optimizer.step() # Atualiza os pesos
        
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        
    train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)

    # --- Validação ---
    model.eval() # Coloca o modelo em modo de avaliação
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad(): # Desativa o cálculo de gradientes
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            
    val_loss = running_val_loss / len(validation_loader)
    val_acc = correct_val / total_val
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f'Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

### 2.3. Gráficos de Acurácia e Perda

Plotamos as métricas salvas durante o treinamento para visualizar o comportamento do modelo.

In [ ]:
epochs_range = range(EPOCHS)

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history['train_acc'], label='Acurácia de Treino')
plt.plot(epochs_range, history['val_acc'], label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history['train_loss'], label='Perda de Treino')
plt.plot(epochs_range, history['val_loss'], label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perda de Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.show()

## 3. Avaliação e Testes

### 3.1. Avaliação do Desempenho no Conjunto de Teste

Usamos o `test_loader` para fazer previsões no conjunto de teste. Coletamos todos os rótulos e previsões para usar as funções do `scikit-learn` e gerar o relatório final.

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).cpu()
        
        predicted = (outputs > 0.5).int()
        
        y_pred.extend(predicted.numpy().flatten())
        y_true.extend(labels.numpy().flatten())

print('\nRelatório de Classificação:')
print(classification_report(y_true, y_pred, target_names=['cats', 'dogs']))

print('\nMatriz de Confusão:')
print(confusion_matrix(y_true, y_pred))

### 3.2. Teste com Novas Imagens

Criamos uma função que carrega uma nova imagem, aplica as mesmas transformações do conjunto de teste e usa o modelo treinado para fazer uma previsão.

In [ ]:
def predict_new_image(image_path, model, device):
    model.eval()
    
    # Carrega e pré-processa a imagem
    image = Image.open(image_path).convert('RGB')
    input_tensor = val_test_transforms(image) # Usa as mesmas transforms da validação
    input_batch = input_tensor.unsqueeze(0) # Cria um batch com uma única imagem
    
    with torch.no_grad():
        output = model(input_batch.to(device))
        prediction_prob = output.item()
    
    # Exibe a imagem e o resultado
    plt.imshow(image)
    plt.axis('off')
    
    class_names = train_dataset.classes
    if prediction_prob > 0.5:
        # Classe 1 (dogs)
        plt.title(f'Previsão: {class_names[1]} ({prediction_prob:.2f})')
    else:
        # Classe 0 (cats)
        plt.title(f'Previsão: {class_names[0]} ({1 - prediction_prob:.2f})')
    plt.show()

# Crie o diretório se não existir
new_images_dir = 'novas_imagens'
os.makedirs(new_images_dir, exist_ok=True)
print(f"Por favor, adicione suas imagens de teste na pasta '{new_images_dir}' e atualize a lista abaixo.")

# ATENÇÃO: Adicione aqui os caminhos para as suas imagens de teste
new_images_paths = [
    # Ex: 'novas_imagens/meu-gato.jpg',
    # Ex: 'novas_imagens/cachorro-vizinho.png'
]

if not new_images_paths:
    print("\nNenhuma imagem nova para testar. Adicione os caminhos à lista 'new_images_paths'.")
else:
    for img_path in new_images_paths:
        if os.path.exists(img_path):
            predict_new_image(img_path, model, device)
        else:
            print(f"Arquivo não encontrado: {img_path}")

## 4. Conclusão Final

Neste exercício, construímos um pipeline de classificação de imagens utilizando **PyTorch**.

**Resumo dos Resultados:**
* **Preparação dos Dados:** A combinação de `torchvision.transforms`, `datasets.ImageFolder` e `DataLoader` provou ser uma maneira robusta e eficiente de carregar, pré-processar e aumentar os dados de imagem para o treinamento.

* **Modelo CNN:** A definição do modelo como uma classe `nn.Module` em PyTorch é clara e estruturada. O loop de treinamento manual, embora mais verboso que o `.fit()` do Keras, nos deu total controle sobre cada etapa do processo, desde a atualização dos gradientes até a avaliação em tempo real.

* **Avaliação:** O modelo alcançou um desempenho sólido no conjunto de teste, demonstrando que a arquitetura e a estratégia de treinamento foram eficazes. O controle granular do PyTorch permite um monitoramento preciso do desempenho em cada etapa.

**Comparação e Próximos Passos:**
A principal diferença em relação à abordagem com TensorFlow/Keras foi a necessidade de escrever o loop de treinamento e validação manualmente. Isso oferece grande flexibilidade para cenários de pesquisa ou implementações customizadas, enquanto Keras oferece uma API de mais alto nível para prototipagem rápida.

Os próximos passos são os mesmos, independentemente do framework:
1.  **Ajuste de Hiperparâmetros:** Testar diferentes otimizadores (`Adam`), taxas de aprendizado e arquiteturas.
2.  **Transfer Learning:** A maior melhoria viria do uso de uma rede pré-treinada do `torchvision.models` (como `resnet50` ou `mobilenet_v2`), que é uma prática padrão em visão computacional para alcançar alta acurácia com menos dados e tempo de treinamento.